### 1* Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (взяв список новостей пользователя)
    - подсказка 1: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал
    - подсказка 2: нужен именно idf, как вес.


_____
_____

импортируем библиотеки:

In [1]:
#!pip install razdel pymorphy2

In [1]:
import pandas as pd
from sklearn.metrics import (f1_score, roc_auc_score, precision_score,
                             classification_report, precision_recall_curve, 
                             confusion_matrix)
import re
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import math
import matplotlib.pyplot as plt

from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from gensim.test.utils import datapath

from razdel import tokenize 
import pymorphy2 
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
tqdm.pandas()
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


качаем файлы:

In [2]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR" -O articles.csv && rm -rf /tmp/cookies.txt 
!wget 'https://drive.google.com/uc?export=download&id=1Q97K9eGrvpbS4ut5CphZa--gJDRqQp2a' -O users_articles.csv
!wget 'https://drive.google.com/uc?export=download&id=1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK' -O users_churn.csv
!wget 'https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c' -O stopwords.txt

--2023-03-21 22:51:54--  https://docs.google.com/uc?export=download&confirm=t&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR
Resolving docs.google.com (docs.google.com)... 172.217.204.100, 172.217.204.113, 172.217.204.102, ...
Connecting to docs.google.com (docs.google.com)|172.217.204.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mt6d08pa8q6ao9515ribq6chjflv8q2g/1679439075000/14904333240138417226/*/1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR?e=download&uuid=4f319657-3d5e-49f4-bd48-53f92ab11183 [following]
--2023-03-21 22:51:54--  https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mt6d08pa8q6ao9515ribq6chjflv8q2g/1679439075000/14904333240138417226/*/1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR?e=download&uuid=4f319657-3d5e-49f4-bd48-53f92ab11183
Resolving doc-0s-c0-docs.googleusercontent.com (doc-0s-c0-docs.googleusercontent.com)... 173.194.214.

ниже код:

In [3]:
def word_processing(df):
    '''обработка текста'''

    def clean_text(text):
        '''очистка текста, на выходе очищеный текст'''
        
        if not isinstance(text, str):
            text = str(text)
        text = text.lower()
        text = text.strip('\n').strip('\r').strip('\t')
        text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))
        text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
        text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
        text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
        text = re.sub('n', ' ', text)
        return text

    cache = {}
    morph = pymorphy2.MorphAnalyzer()

    def lemmatization(text):    
        '''лемматизация, на выходе лист лемматизированых токенов'''

        # w - стоп-слова
        w = stopwords.words('russian')
        with open('stopwords.txt') as f:
            additional_stopwords = [w.strip() for w in f.readlines() if w]
        w += additional_stopwords 
        
        # если зашел тип не `str` делаем его `str`
        if not isinstance(text, str):
            text = str(text)
        # токенизация предложения через razdel
        tokens = list(tokenize(text))
        words = [_.text for _ in tokens]

        words_lem = []
        for w in words:
            if w[0] == '-': # проверка есть ли в начале слова '-'
                w = w[1:]
            if len(w) > 1: # проверка токена с одного символа
                if w in cache: # проверка есть ли данное слово в кэше
                    words_lem.append(cache[w])
                else: # лемматизация слова
                    temp_cach = cache[w] = morph.parse(w)[0].normal_form
                    words_lem.append(temp_cach)
        # проверка на стоп-слова:
        words_lem_without_stopwords = [i for i in words_lem if not i in w] 
        return words_lem_without_stopwords
    df['title'] = df['title'].progress_apply(lambda x: clean_text(x))
    df['title'] = df['title'].progress_apply(lambda x: lemmatization(x))

    return df['title']

In [4]:
def trained_model(df, N_topic=20, passes=2):
    '''создание модели'''
    
    # сформируем список наших текстов
    texts = list(news['title'].values)
    # Создадим корпус из списка с текстами
    common_dictionary = Dictionary(texts)
    common_corpus = [common_dictionary.doc2bow(text) for text in texts]
    # Обучаем модель на корпусе
    lda = LdaModel(common_corpus, num_topics=N_topic, 
                   id2word=common_dictionary, passes=passes)
    # Сохраняем модель на диск
    temp_file = datapath("model.lda")
    lda.save(temp_file)
    # Загружаем обученную модель с диска
    return common_dictionary, LdaModel.load(temp_file)

In [5]:
def documents_to_vector(df, lda, common_dictionary, N_topic=15):
    '''функция возвращает датафрейм из вероятностей новостей'''

    def get_lda_vector(lda, text):
        '''функция возвращает векторное представление новости'''
        
        unseen_doc = common_dictionary.doc2bow(text)
        lda_tuple = lda[unseen_doc]

        not_null_topics = dict(zip([i[0] for i in lda_tuple], 
                                  [i[1] for i in lda_tuple]))
        output_vector = []
        for i in range(N_topic):
            if i not in not_null_topics:
                output_vector.append(0)
            else:
                output_vector.append(not_null_topics[i])
        return np.array(output_vector)

    topic_matrix = pd.DataFrame([get_lda_vector(lda, text) for text in df['title'].values])
    topic_matrix.columns = [f'topic_{i}' for i in range(N_topic)]
    topic_matrix['doc_id'] = df['doc_id'].values
    topic_matrix = topic_matrix[['doc_id']+[f'topic_{i}' for i in range(N_topic)]]
    return topic_matrix


In [6]:
def get_best_scores():
    '''функция возвращает метрики, взвешивая новости по tfidf, работает с параметрами глобальных переменных'''

    def get_user_embedding_idf(user_news_list, doc_dict, func=np.median):
        
        user_news_list = eval(user_news_list)
        user_vector = np.zeros((len(user_news_list), N_topic))
        for i, doc_id in enumerate(user_news_list):
            # на случай, если новости не окажется
            try:
                weight = idf[idf['article'] == str(doc_id)]['idf'].values[0]
            except Exception as e:
                weight = 0
            # документ умножаем на его вес
            user_vector[i] = doc_dict[doc_id] * weight
        user_vector = func(user_vector, axis=0)
        return user_vector

    doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[[f'topic_{i}' for i in range(N_topic)]].values))
    # готовим строку - номер статьи будет выступать как слово
    users['articles_string'] = users['articles'].apply(lambda x: x.replace('[','').replace(']', '').replace(',', ''))
    # обучаемся на модели и забираем только idf, так как нам важен именно обратный вес по документу
    tfidf = TfidfVectorizer()
    tfidf.fit(users['articles_string'])
    idf = pd.DataFrame({'article': tfidf.get_feature_names_out(), 'idf': tfidf.idf_})

    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_idf(x, doc_dict))])
    user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]

    X = pd.merge(user_embeddings, target, 'left')
    X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], X['churn'], random_state=0)

    model = LogisticRegression()
    model.fit(X_train, y_train)
    preds = model.predict_proba(X_test)[:, 1]

    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    ix = np.argmax(fscore)
    roc_auc = roc_auc_score(y_test, preds)

    return thresholds[ix], fscore[ix], precision[ix], recall[ix], roc_auc

загружаем файлы:

In [7]:
news = pd.read_csv("articles.csv")
users = pd.read_csv("users_articles.csv")
target = pd.read_csv("users_churn.csv")

подготавливаем текст:

In [9]:
news['title'] = word_processing(news)

100%|██████████| 27000/27000 [03:12<00:00, 139.93it/s]


создаём LDA-модель (passes - количество проходов):

In [10]:
N_topic = 20
common_dictionary, lda = trained_model(news, N_topic=N_topic, passes=5)

# формируем матрицу вероятностей для новостей
topic_matrix = documents_to_vector(news, lda, common_dictionary, N_topic=N_topic)

In [11]:
thresholds, fscore, precision, recall, roc_auc = get_best_scores()

In [12]:
print(f'{thresholds=:.2f}\n{fscore=:.2f}\n{precision=:.2f}\n{recall=:.2f}\n{roc_auc=:.2f}')

thresholds=0.35
fscore=0.87
precision=0.83
recall=0.91
roc_auc=0.99


_____
_____

### Сформировать на выходе единую таблицу, сравнивающую качество 2/3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
### Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

|  | thresholds | F-Score | precision | recall | ROC-auc |
|-------------|-------------|-------------|-------------|-------------|-------------|
| mean  | 0.4  | 0.74  | 0.71  | 0.78  | 0.97  |
| median  | 0.47  | 0.83  | 0.87  | 0.84  | 0.98  |
| max  | 0.41  | 0.79  | 0.82  | 0.81  | 0.97  |
| idf median  | 0.35  | 0.87  | 0.83  | 0.91  | 0.99  |


**Вывод**: в работе исследовались разные гиперпараметры для модели:
- количеств тем от 5-ти до 20-ти
- количесвтво проходов от 1 до 5-ти
- виды агрегации: среднее, медиана, максимум

Практически всегда учшие показатели модель даёт при 20-ти темах и 5-ти проходах (только при максимуме лучшим результат был при 3-х проходах)

При 5-ти проходах для любой агрегации метрика модель даёт хороший результат, но медиана показала себя лучше всех, так как она менее чувствительна к выбросам.

А алгоритм со взвешенными документами и медианой показал себя лучше всех, так как не все новости одинаково важны, а он добавляет вес уникальным. Аналогичные "другим" показатели этот алгоритм даёт уже при 10-ти новостях, а при 20-ти опережает "других" по всем метрикам, это видно в сводной таблице